In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()

len(ticker_lst)

62

In [3]:
# define function call_tickers that takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_ticker_lst = []
    # write a for loop to iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_ticker_lst.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_ticker_lst

# defein function filter_USD that takes a list of called tickers and return a list of only US listed tickers
def filter_USD(lst):
    # create a new list to store the us tickers
    usd_lst = []
    # write a for loop that iterates the called ticker list
    for i in lst:
        # use try and except to ignore the delisted tickers
        try:
            # if the currency of the stock is "USD"
            if i.info['currency'] == 'USD':
                # append the ticker to the usd list
                usd_lst.append(i)
        except:
            pass
    # return the usd list
    return usd_lst

# define function filter_daily_volumn that takes a called ticker list, start date, end date, 
# and returns a list of tickers with an average daily_volume of at least 10000 shares in that time period
def filter_daily_volume(lst,start,end):
    # write a for loop to iterate the called ticker list
    for i in lst:
        # extract the volume of the stock in the given time period and see if the avergae is less than 10000
        if i.history(start = start, end = end,interval='1d')['Volume'].mean() < 10000:
            # if yes, remove the ticker from the list
            lst.remove(i)
        # return the list with unqualified tickers removed
        return lst


    
def get_prices(lst,start,end):
    prices = pd.DataFrame()
    for i in lst:
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    return prices

def get_std(lst):
    std = pd.DataFrame(columns = ('ticker','std'))
    for i in range(len(lst)):
        std.loc[i] = [str(lst[i])[24:-1], prices[str(lst[i])[24:-1]].std()]
    return std

def rank(df,column_name,method):
    df['rank'] = df[column_name].rank(method=method)
    df = df.sort_values(by='rank')
    return df

In [4]:
# Getting the Symbol (Ticker) of each Stock
def get_ticker_Symbol (lst):
    
    ticker_lst = []
    for ticker in lst:
        ticker_lst.append(ticker.info['symbol'])
    return ticker_lst

In [5]:
# Calculating Beta
def get_beta(good_tickers, prices, start_date, end_date, ticker_symbols):
    Ticker = '^GSPC'
    MarketIndex = yf.Ticker(Ticker) # The symbol yfinance uses for the S&P 500

    MarketIndex_hist = MarketIndex.history(start=start_date, end=end_date)

    # DataFrame for Market Index
    marketDF = pd.DataFrame(MarketIndex_hist['Close'])
    marketDF.columns = [Ticker]
    

    # Loop iterates through the column of prices
    betaList = []
    for i in range(len(ticker_symbols)):
        currentStockPrice = prices[ticker_symbols[i]]
        currentStockPrice = pd.concat([currentStockPrice, marketDF], join = 'inner', axis=1)
        
        # Getting the Monthly Return of each stock
        monthly_returns = currentStockPrice.resample('MS').first().pct_change()  # Dropping the first entry (since it's N/A)
        monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
        
        
        # Calculate the market variance (you will need to reference the column correponding to the market)
        MarketVar = monthly_returns[Ticker].var()
        
        betaList.append(monthly_returns.cov() / MarketVar)
        
    # Filter through betaList and extract the beta for each stock
    for i in range(len(betaList)):
        betaList[i] = betaList[i].iat[0,1]
    
    # Creating a DataFrame for Tickers and their Beta value
    beta = pd.DataFrame(columns = ('ticker','beta'))
    for i in range(len(good_tickers)):
        beta.loc[i] = [str(good_tickers[i])[24:-1], betaList[i]]

    return beta

In [6]:
start_date = "2021-07-02"
end_date = "2021-10-22"


# call the tickers
called_tickers = call_tickers(ticker_lst)

# filter the usd tickers
usd_tickers = filter_USD(called_tickers)

# filter the tickers with required average daily volumn
good_tickers = filter_daily_volume(usd_tickers, start_date, end_date)

# Get a list of Ticker Symbols (list of Strings)
ticker_symbols = get_ticker_Symbol (good_tickers)

# extract the closing prices for the good tickers
prices = get_prices(good_tickers, start_date, end_date)

# calculate the std according to the closing prices
std = get_std(good_tickers)

# rank them in ascending order
std = rank(std,'std','min')

# calculate the beta according to the closing prices
beta = get_beta(good_tickers, prices, start_date, end_date, ticker_symbols)

# rank them in ascending order
beta = rank(beta,'beta','min')

In [7]:
std.head()

,ticker,std,rank
30,MON,0.046874,1.0
48,T,0.539296,2.0
25,KMI,0.760351,3.0
29,MO,1.189154,4.0
26,KO,1.205896,5.0


In [8]:
beta.head()

,ticker,beta,rank
18,COP,-4.940763,1.0
37,OXY,-4.786461,2.0
22,GM,-3.146237,3.0
45,SLB,-1.906580,4.0
25,KMI,-1.738541,5.0


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.